<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Modification of the fuselage length of an aircraft.

This notebook is based on the 3rd FAST-OAD-GA tutorial written by Aurélien. It aims to compare a reference architecture represented by the .XML file of an aircraft (Beechcraft 76, Cirrus 22, TBM-900) with a modified configuration. The modified architecture is produced by the FAST-OAD-GA module called modify_config which creates the corresponding .XML file. Then the .xml files of the reference aircraft and of the modified one go through all of the FAST-OAD-GA modules, with the generate_block_analysis function which calls each module. The variable viewer is called for both aircrafts after each module.

The modification implemented in this notebook is the increase of the fuselage length in order to add a row of passengers. The user has to define the multiplier of the span, and can also play on other parameters such as setting the engine position along the span.

The fastga code is from the root analysis_mode on GitHub.

Like in the tutorials, the structure of the notebook is based on data transfers between the data folder which stocks the .xml of the reference aircraft, and the workdir folder which stocks the .xml files passing through the modules of FAST-OAD-GA. After all the modules have been executed the final .xml files are placed in the output folder.

As the TBM-900 turbopropeller is still being modelized in FAST-OAD-GA, the Beechcraft 76 will be used for this analysis. There is no limit in the code for the increase in span, but some modules could have problems running with an extreme value (such as two times the original span).

## 1. Initialization of the work environment

### Choice of the aircraft

In [1]:
from ipywidgets import widgets, Layout
from fastoad.io import VariableIO
from IPython.display import display, clear_output

liste_aircraft = widgets.Dropdown(
    options=[('Beechcraft Duchess 76', 'output_mda_beech.xml'), ('Cirrus SR22', 'output_mda_cirrus_sr22.xml'), ('TBM-900', 'output_mda_tbm900.xml')],
    value='output_mda_beech.xml',
    description='Reference Aircraft:',
    style={'description_width': 'initial'},
    layout=Layout(width='30%'),
)

# Initialization of the reference aircraft
ref_aircraft = 'output_mda_beech.xml'
print('The reference aircraft .xml file is '+ ref_aircraft)

def eventhandler(change):
    if (change.new):
        ref_aircraft = liste_aircraft.value
        print('The reference aircraft .xml file is '+ ref_aircraft)

liste_aircraft.observe(eventhandler, names='value')

display(liste_aircraft)

Failed to import module fastga.models.load_analysis.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.performances.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.geometry.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.performances.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_beechcraft_76.py
Failed to import module fastga

The reference aircraft .xml file is output_mda_beech.xml


Dropdown(description='Reference Aircraft:', layout=Layout(width='30%'), options=(('Beechcraft Duchess 76', 'ou…

### Choice of the engine model

In [2]:
liste_engine = widgets.Dropdown(
    options=[('Basic IC Engine', "fastga.wrapper.propulsion.basicIC_engine"), ('Basic Turboprop', "fastga.wrapper.propulsion.basic_turboprop")],
    value="fastga.wrapper.propulsion.basicIC_engine",
    description='Propulsion Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='30%'),
)

# Initialization of the engine model
engine_id = "fastga.wrapper.propulsion.basicIC_engine"
print('The engine model id chosen is '+ engine_id)

def eventhandler(change):
    if (change.new):
        engine_id = liste_engine.value
        print('The engine model id chosen is '+ engine_id)

liste_engine.observe(eventhandler, names='value')

display(liste_engine)

The engine model id chosen is fastga.wrapper.propulsion.basicIC_engine


Dropdown(description='Propulsion Model:', layout=Layout(width='30%'), options=(('Basic IC Engine', 'fastga.wra…

### Preparation of files

In [3]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import fastga
import logging
from fastoad.gui import VariableViewer
from fastoad.io import VariableIO
import shutil

# Define relative path.
current_path = os.getcwd()
DATA_FOLDER_PATH = pth.join(current_path, 'data')
WORK_FOLDER_PATH = pth.join(current_path, 'workdir')
OUTPUT_FOLDER_PATH = pth.join(current_path, 'output')

# Clear work folder.
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
os.mkdir(WORK_FOLDER_PATH)

# For using more screen width.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Final file names. The .xml files in the work folder will be overwritten each time they pass through a module.
CONFIG_FILE = pth.join(DATA_FOLDER_PATH, 'weight_loop.yml')
WORK_CONFIG_FILE = pth.join(WORK_FOLDER_PATH, 'weight_loop.yml')
AIRCRAFT_REF_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_ref.xml')
AIRCRAFT_MOD_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_mod.xml')
WEIGHT_LOOP_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs_weight_loop.xml')

# Copy the reference aircraft .xml file.
shutil.copy(pth.join(OUTPUT_FOLDER_PATH, ref_aircraft), AIRCRAFT_REF_FILE)
shutil.copy(AIRCRAFT_REF_FILE, AIRCRAFT_MOD_FILE)
shutil.copy(CONFIG_FILE, WORK_CONFIG_FILE)

'C:\\Users\\Lucas\\FAST-OAD-GA\\FAST-GA\\src\\fastga\\notebooks\\Lucas\\workdir\\weight_loop.yml'

## 2. Definition of the fuselage modifications from the reference architecture

Describes the different parameters the user has to choose in order to obtain the desired modified configuration. The function generate_block_analysis is then used twice to create the .xml file related to the modified configuration.

The same parameters have to be used for the ComputeConfigMod class and for the UpdateXML class.

In [ ]:
# Fuselage modifications
added_length = 0    # If 0, the length of a row of seats will be taken
added_section_x_ratio_front = 0
added_section_x_ratio_rear = 1
added_pax = 2     # Integer stating the number of passengers added in the design mission. It will be added to the npax_design of the xml, so take care not to exceed the new npax_max.
added_luggage = 10     # float stating the mass of luggage added in the design mission

fuselage_mod_parameters = [added_length, added_section_x_ratio_front, added_section_x_ratio_rear, added_pax, added_luggage]

## 3. Computation of the modified architecture

In [ ]:
from fastga.models.modify_config import ComputeConfigMod
from fastga.models.modify_config.update_XML import UpdateXML
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.weight.weight import Weight
from fastga.models.weight.mass_breakdown.update_mtow import UpdateMTOW
from fastga.models.performances.mission import Mission
from fastga.models.load_analysis.loads import Loads
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities
from fastga.models.aerodynamics.external.openvsp.compute_vn import ComputeVNopenvsp


# Compute the modifications
compute_modify = api_cs23.generate_block_analysis(
        ComputeConfigMod(fuselage_mod=fuselage_mod_parameters),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_modify({})

compute_update = api_cs23.generate_block_analysis(
        UpdateXML(fuselage_mod=fuselage_mod_parameters),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_update({})

# GEOMETRY
compute_geometry_mod = api_cs23.generate_block_analysis(
        Geometry(propulsion_id=engine_id),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_geometry_mod({})

# AERODYNAMICS
compute_aero_mod = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id=engine_id,
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=True,
#             wing_airfoil_file="naca43013_3.af"
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_aero_mod({})

# WEIGHT / MTOW / PERFORMANCES loop

shutil.copy(AIRCRAFT_MOD_FILE, WEIGHT_LOOP_FILE)

eval_problem = api_cs25.evaluate_problem(WORK_CONFIG_FILE, overwrite=True)

OUTPUT_FILE = pth.join(WORK_FOLDER_PATH,'problem_outputs_weight_loop.xml')
shutil.copy(OUTPUT_FILE, AIRCRAFT_MOD_FILE)

# LOAD ANALYSIS
compute_loads_mod = api_cs23.generate_block_analysis(
        Loads(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_loads_mod({})

# HANDLING QUALITIES
compute_hq_mod = api_cs23.generate_block_analysis(
        ComputeHandlingQualities(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_hq_mod({})

# VN
compute_vn = api_cs23.generate_block_analysis(
        ComputeVNopenvsp(),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output = compute_vn({})

In [ ]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')
shutil.copy(AIRCRAFT_REF_FILE, OUTPUT_REF_FILE)
shutil.copy(AIRCRAFT_MOD_FILE, OUTPUT_MOD_FILE)

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot

# Aircraft geometry plot
fig = aircraft_geometry_plot(OUTPUT_REF_FILE, name='reference', plot_nacelle = False)
fig = aircraft_geometry_plot(OUTPUT_MOD_FILE, name='modified fuse', plot_nacelle = False, fig=fig)
fig.show()

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 4. Analysis of the wing span increase

In [ ]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')
AIRCRAFT_SPAN_FILE = pth.join(WORK_FOLDER_PATH, 'aircraft_span_mod.xml')

In [ ]:
# Span modifications
span_multiplier_array = [1.1, 1.2]
output_span_path_array = []

fixed_engine_position = True       # boolean stating if the engines stay at the same y-position along the span or if their y-ratio is conserved
fuel_added_part = False            # boolean stating if fuel tanks are added in the extended part of the wing. If True, only applies for y_ratio_tank_end =1.0 in the initial .xml file.

In [ ]:
from fastga.models.modify_config import ComputeConfigMod
from fastga.models.modify_config.update_XML import UpdateXML
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.weight.weight import Weight
from fastga.models.weight.mass_breakdown.update_mtow import UpdateMTOW
from fastga.models.performances.mission import Mission
from fastga.models.load_analysis.loads import Loads
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities
from fastga.models.aerodynamics.external.openvsp.compute_vn import ComputeVNopenvsp


for span_multiplier in span_multiplier_array:

    shutil.copy(OUTPUT_MOD_FILE, AIRCRAFT_SPAN_FILE)
    
    # Modify and update geometry
    compute_modify = api_cs23.generate_block_analysis(
            ComputeConfigMod(span_mod=[span_multiplier, fixed_engine_position, fuel_added_part]),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_modify({})

    compute_update = api_cs23.generate_block_analysis(
        UpdateXML(span_mod=[span_multiplier, fixed_engine_position, fuel_added_part]),
        [],
        str(AIRCRAFT_SPAN_FILE),
        True,
    )
    output = compute_update({})
    
    # GEOMETRY
    compute_geometry = api_cs23.generate_block_analysis(
            Geometry(propulsion_id=engine_id),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_geometry({})

    # AERODYNAMICS
    compute_aero = api_cs23.generate_block_analysis(
            Aerodynamics(
                propulsion_id=engine_id,
                use_openvsp=True,
                compute_mach_interpolation=True,
                compute_slipstream_cruise=True,
    #             wing_airfoil_file="naca43013_3.af"
            ),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_aero({})

    # WEIGHT / MTOW / PERFORMANCES loop for modified aircraft
    shutil.copy(AIRCRAFT_SPAN_FILE, WEIGHT_LOOP_FILE)

    eval_problem = api_cs25.evaluate_problem(WORK_CONFIG_FILE, overwrite=True)

    OUTPUT_FILE = pth.join(WORK_FOLDER_PATH,'problem_outputs_weight_loop.xml')
    shutil.copy(OUTPUT_FILE, AIRCRAFT_SPAN_FILE)

    # LOAD ANALYSIS
    compute_loads = api_cs23.generate_block_analysis(
            Loads(),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_loads({})
    
    # HANDLING QUALITIES
    compute_hq = api_cs23.generate_block_analysis(
            ComputeHandlingQualities(
                propulsion_id=engine_id,
            ),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_hq({})
    
    # VN
    compute_vn = api_cs23.generate_block_analysis(
            ComputeVNopenvsp(),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_vn({})
    
    output_file_name = 'output_span_' + str(span_multiplier).replace('.','_') + '.xml'
    shutil.copy(AIRCRAFT_SPAN_FILE, pth.join(OUTPUT_FOLDER_PATH, output_file_name))
    output_span_path_array.append(pth.join(OUTPUT_FOLDER_PATH, output_file_name))

### Post Processing

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import force_repartition_diagram, shear_diagram, rbm_diagram
from fastga.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot
from fastga.utils.postprocessing.analysis_and_plots_vn import evolution_diagram


# Aircraft geometry plot
fig = aircraft_geometry_plot(OUTPUT_REF_FILE, name='reference', plot_nacelle = False)
fig = aircraft_geometry_plot(OUTPUT_MOD_FILE, name='modified fuse', plot_nacelle = False, fig=fig)

for i in range(len(span_multiplier_array)):
    fig = aircraft_geometry_plot(output_span_path_array[i], name='wing span multiplied by ' + str(span_multiplier_array[i]), plot_nacelle = False, fig=fig)

fig.show()


# Force repartition diagram
fig = force_repartition_diagram(OUTPUT_REF_FILE, name='reference')
fig = force_repartition_diagram(OUTPUT_MOD_FILE, name='modified fuse',fig=fig)

for i in range(len(span_multiplier_array)):
    fig = force_repartition_diagram(output_span_path_array[i], name='wing span multiplied by ' + str(span_multiplier_array[i]), fig=fig)

fig.show()


# Shear diagram
fig = shear_diagram(OUTPUT_REF_FILE, name='reference')
fig = shear_diagram(OUTPUT_MOD_FILE, name='modified fuse',fig=fig)

for i in range(len(span_multiplier_array)):
    fig = shear_diagram(output_span_path_array[i], name='wing span multiplied by ' + str(span_multiplier_array[i]), fig=fig)

fig.show()


# Root bending moment diagram
fig = rbm_diagram(OUTPUT_REF_FILE, name='reference')
fig = rbm_diagram(OUTPUT_MOD_FILE, name='modified fuse',fig=fig)

for i in range(len(span_multiplier_array)):
    fig = rbm_diagram(output_span_path_array[i], name='wing span multiplied by ' + str(span_multiplier_array[i]), fig=fig)

fig.show()


# Manoeuver Diagram
fig = evolution_diagram(OUTPUT_REF_FILE, name='reference aircraft')
fig = evolution_diagram(OUTPUT_MOD_FILE, name='fuselage mod', fig=fig)

for i in range(len(span_multiplier_array)):
    fig = evolution_diagram(output_span_path_array[i], name='wing span multiplied by ' + str(span_multiplier_array[i]), fig=fig)

fig.show()


# Static margins
variables_ref = VariableIO(OUTPUT_REF_FILE).read()
static_margin_fixed_ref = round(variables_ref["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_ref = round(variables_ref["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("reference aircraft\t\t", "static margin stick fixed:", static_margin_fixed_ref, "\tstatic margin stick free:", static_margin_free_ref)

variables_mod = VariableIO(OUTPUT_MOD_FILE).read()
static_margin_fixed_mod = round(variables_mod["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_mod = round(variables_mod["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("fuselage extended aircraft\t", "static margin stick fixed:", static_margin_fixed_mod, "\tstatic margin stick free:", static_margin_free_mod)

for i in range(len(span_multiplier_array)):
    variables = VariableIO(output_span_path_array[i]).read()
    static_margin_fixed = round(variables["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
    static_margin_free = round(variables["data:handling_qualities:stick_free_static_margin"].value[0],4)
    print("wing span multiplied by", span_multiplier_array[i], "\t", "static margin stick fixed:", static_margin_fixed, "\tstatic margin stick free:", static_margin_free)

## 5. Analysis of the wing longitudinal position modification

In [ ]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')
AIRCRAFT_WING_POS_FILE = pth.join(WORK_FOLDER_PATH, 'aircraft_wing_pos_mod.xml')

In [ ]:
variables_mod = VariableIO(OUTPUT_MOD_FILE).read()
wing_mac_25 = variables_mod["data:geometry:wing:MAC:at25percent:x"].value[0]
htp_25 = variables_mod["data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25"].value[0]

delta_wing_array = [0.5, 1]
output_position_path_array = []

In [ ]:
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.weight.weight import Weight
from fastga.models.weight.mass_breakdown.update_mtow import UpdateMTOW
from fastga.models.performances.mission import Mission
from fastga.models.load_analysis.loads import Loads
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities
from fastga.models.aerodynamics.external.openvsp.compute_vn import ComputeVNopenvsp

for delta_wing in delta_wing_array:
    
    shutil.copy(OUTPUT_MOD_FILE, AIRCRAFT_WING_POS_FILE)
    
    var_inputs = ["data:geometry:wing:MAC:at25percent:x", "data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25"]

    # GEOMETRY
    compute_geometry = api_cs23.generate_block_analysis(
            Geometry(propulsion_id=engine_id),
            var_inputs,
            str(AIRCRAFT_WING_POS_FILE),
            True,
        )
    
    inputs_dict = {"data:geometry:wing:MAC:at25percent:x":(wing_mac_25 + delta_wing, "m"), "data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25":(htp_25 - delta_wing, "m")}
    output = compute_geometry(inputs_dict)
    
    # AERODYNAMICS
    compute_aero = api_cs23.generate_block_analysis(
            Aerodynamics(
                propulsion_id=engine_id,
                use_openvsp=True,
                compute_mach_interpolation=True,
                compute_slipstream_cruise=True,
    #             wing_airfoil_file="naca43013_3.af"
            ),
            [],
            str(AIRCRAFT_WING_POS_FILE),
            True,
        )
    output = compute_aero({})

    # WEIGHT / MTOW / PERFORMANCES loop
    shutil.copy(AIRCRAFT_WING_POS_FILE, WEIGHT_LOOP_FILE)

    eval_problem = api_cs25.evaluate_problem(WORK_CONFIG_FILE, overwrite=True)

    OUTPUT_FILE = pth.join(WORK_FOLDER_PATH,'problem_outputs_weight_loop.xml')
    shutil.copy(OUTPUT_FILE, AIRCRAFT_WING_POS_FILE)
    
    # LOAD ANALYSIS
    compute_loads = api_cs23.generate_block_analysis(
            Loads(),
            [],
            str(AIRCRAFT_WING_POS_FILE),
            True,
        )
    output = compute_loads({})

    # HANDLING QUALITIES
    compute_hq = api_cs23.generate_block_analysis(
            ComputeHandlingQualities(
                propulsion_id=engine_id,
            ),
            [],
            str(AIRCRAFT_WING_POS_FILE),
            True,
        )
    output = compute_hq({})
    
    # VN
    compute_vn = api_cs23.generate_block_analysis(
            ComputeVNopenvsp(),
            [],
            str(AIRCRAFT_WING_POS_FILE),
            True,
        )
    output = compute_vn({})
    
    output_file_name = 'output_wing_pos_delta_' + str(delta_wing).replace('.','_') + '.xml'
    shutil.copy(AIRCRAFT_WING_POS_FILE, pth.join(OUTPUT_FOLDER_PATH, output_file_name))
    output_position_path_array.append(pth.join(OUTPUT_FOLDER_PATH, output_file_name))

### Post Processing

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import force_repartition_diagram, shear_diagram, rbm_diagram
from fastga.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot
from fastga.utils.postprocessing.analysis_and_plots_vn import evolution_diagram


# Aircraft geometry plot
fig = aircraft_geometry_plot(OUTPUT_REF_FILE, name='reference', plot_nacelle = False)
fig = aircraft_geometry_plot(OUTPUT_MOD_FILE, name='modified fuse', plot_nacelle = False, fig=fig)

for i in range(len(delta_wing_array)):
    fig = aircraft_geometry_plot(output_position_path_array[i], name='wing position moved by ' + str(delta_wing_array[i]), plot_nacelle = False, fig=fig)

fig.show()


# Force repartition diagram
fig = force_repartition_diagram(OUTPUT_REF_FILE, name='reference')
fig = force_repartition_diagram(OUTPUT_MOD_FILE, name='modified fuse',fig=fig)

for i in range(len(delta_wing_array)):
    fig = force_repartition_diagram(output_position_path_array[i], name='wing position moved by ' + str(delta_wing_array[i]) + 'm', fig=fig)

fig.show()


# Shear diagram
fig = shear_diagram(OUTPUT_REF_FILE, name='reference')
fig = shear_diagram(OUTPUT_MOD_FILE, name='modified fuse',fig=fig)

for i in range(len(delta_wing_array)):
    fig = shear_diagram(output_position_path_array[i], name='wing position moved by ' + str(delta_wing_array[i]) + 'm', fig=fig)

fig.show()


# Root bending moment diagram
fig = rbm_diagram(OUTPUT_REF_FILE, name='reference')
fig = rbm_diagram(OUTPUT_MOD_FILE, name='modified fuse',fig=fig)

for i in range(len(delta_wing_array)):
    fig = rbm_diagram(output_position_path_array[i], name='wing position moved by ' + str(delta_wing_array[i]) + 'm', fig=fig)

fig.show()


# Manoeuver Diagram
fig = evolution_diagram(OUTPUT_REF_FILE, name='reference aircraft')
fig = evolution_diagram(OUTPUT_MOD_FILE, name='fuselage mod', fig=fig)

for i in range(len(delta_wing_array)):
    fig = evolution_diagram(output_position_path_array[i], name='wing position moved by ' + str(delta_wing_array[i]) + 'm', fig=fig)

fig.show()


# Static margins
variables_ref = VariableIO(OUTPUT_REF_FILE).read()
static_margin_fixed_ref = round(variables_ref["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_ref = round(variables_ref["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("reference aircraft\t\t", "static margin stick fixed:", static_margin_fixed_ref, "\tstatic margin stick free:", static_margin_free_ref)

variables_mod = VariableIO(OUTPUT_MOD_FILE).read()
static_margin_fixed_mod = round(variables_mod["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_mod = round(variables_mod["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("fuselage extended aircraft\t", "static margin stick fixed:", static_margin_fixed_mod, "\tstatic margin stick free:", static_margin_free_mod)

for i in range(len(delta_wing_array)):
    variables = VariableIO(output_position_path_array[i]).read()
    static_margin_fixed = round(variables["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
    static_margin_free = round(variables["data:handling_qualities:stick_free_static_margin"].value[0],4)
    print("wing position moved by", delta_wing_array[i], 'm', "\t", "static margin stick fixed:", static_margin_fixed, "\tstatic margin stick free:", static_margin_free)

## 6. Analysis of the static margins by modifying the wing position

In [11]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')

variables_ref = VariableIO(OUTPUT_REF_FILE).read()
static_margin_fixed_ref = round(variables_ref["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_ref = round(variables_ref["data:handling_qualities:stick_free_static_margin"].value[0],4)

In [12]:
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.weight.weight import Weight
from fastga.models.weight.mass_breakdown.update_mtow import UpdateMTOW
from fastga.models.performances.mission import Mission
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities

from scipy.optimize import fsolve

delta_wing_initial = 0.5

def static_margins(
    delta_wing, file_formatter=None
):

    temp = pth.join(current_path, 'temp.xml')
    shutil.copy(OUTPUT_MOD_FILE, temp)

    variables_ref = VariableIO(OUTPUT_MOD_FILE, file_formatter).read()
    
    wing_mac_25 = variables_ref["data:geometry:wing:MAC:at25percent:x"].value[0]
    htp_25 = variables_ref["data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25"].value[0]
    static_margin_ref = variables_ref["data:handling_qualities:stick_fixed_static_margin"].value[0]
    
    var_inputs = ["data:geometry:wing:MAC:at25percent:x",
                  "data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25"]

    # GEOMETRY
    compute_geometry = api_cs23.generate_block_analysis(
        Geometry(propulsion_id=engine_id),
        var_inputs,
        str(temp),
        True,
    )

    inputs_dict = {"data:geometry:wing:MAC:at25percent:x": (wing_mac_25 + delta_wing, "m"),
                   "data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25": (htp_25 - delta_wing, "m")}
    output = compute_geometry(inputs_dict)

    # AERODYNAMICS
    compute_aero = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id=engine_id,
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=True,
        ),
        [],
        str(temp),
        True,
    )
    output = compute_aero({})

    # WEIGHT / MTOW / PERFORMANCES loop
    shutil.copy(temp, WEIGHT_LOOP_FILE)

    eval_problem = api_cs25.evaluate_problem(WORK_CONFIG_FILE, overwrite=True)

    OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_weight_loop.xml')
    shutil.copy(OUTPUT_FILE, temp)

    # HANDLING QUALITIES
    compute_hq = api_cs23.generate_block_analysis(
        ComputeHandlingQualities(
            propulsion_id=engine_id,
        ),
        [],
        str(temp),
        True,
    )
    output = compute_hq({})

    variables_computed = VariableIO(temp, file_formatter).read()

    static_margin_computed = variables_computed["data:handling_qualities:stick_fixed_static_margin"].value[0]

    return static_margin_ref * 1.1 - static_margin_computed


delta_wing = fsolve(static_margins, delta_wing_initial)
print(delta_wing)

NL: NLBGS 1 ; 180005333 1
NL: NLBGS 2 ; 7984.92822 4.4359398e-05
NL: NLBGS 3 ; 2388.09227 1.32667862e-05
NL: NLBGS 4 ; 629.005587 3.49437195e-06
NL: NLBGS 5 ; 160.636577 8.92398987e-07
NL: NLBGS 6 ; 40.6882855 2.26039333e-07
NL: NLBGS 7 ; 10.260205 5.69994499e-08
NL: NLBGS 8 ; 2.58587591 1.43655516e-08
NL: NLBGS 9 ; 0.651632974 3.62007593e-09
NL: NLBGS Converged
NL: NLBGS 1 ; 180005333 1
NL: NLBGS 2 ; 7984.92822 4.4359398e-05
NL: NLBGS 3 ; 2388.09227 1.32667862e-05
NL: NLBGS 4 ; 629.005587 3.49437195e-06
NL: NLBGS 5 ; 160.636577 8.92398987e-07
NL: NLBGS 6 ; 40.6882855 2.26039333e-07
NL: NLBGS 7 ; 10.260205 5.69994499e-08
NL: NLBGS 8 ; 2.58587591 1.43655516e-08
NL: NLBGS 9 ; 0.651632974 3.62007593e-09
NL: NLBGS Converged
NL: NLBGS 1 ; 180005333 1
NL: NLBGS 2 ; 7984.92822 4.4359398e-05
NL: NLBGS 3 ; 2388.09227 1.32667862e-05
NL: NLBGS 4 ; 629.005587 3.49437195e-06
NL: NLBGS 5 ; 160.636577 8.92398987e-07
NL: NLBGS 6 ; 40.6882855 2.26039333e-07
NL: NLBGS 7 ; 10.260205 5.69994499e-08
NL: NL